In [1]:
import findspark

In [2]:
findspark.init('/usr/local/spark')

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Python dataframe example").getOrCreate()

In [11]:
customerDF = spark.read.load("/home/hduser/Desktop/customer.txt",format = "csv",sep = "\t",inferSchema="true",header="true")

In [12]:
customerDF.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [13]:
customerDF.show()

+-----------+----------------+-------------+--------------+----------------+
|customer_id|   customer_name|customer_city|customer_state|customer_zipcode|
+-----------+----------------+-------------+--------------+----------------+
|      11039|     Mary Torres|       Caguas|            PR|             725|
|       5623|      Jose Haley|     Columbus|            OH|           43207|
|       5829|      Mary Smith|      Houston|            TX|           77015|
|       6336|  Richard Maddox|       Caguas|            PR|             725|
|       1708|  Margaret Booth|    Arlington|            TX|           76010|
|      10227|  Mary Henderson|       Caguas|            PR|             725|
|        839|     Lisa Walker|       Caguas|            PR|             725|
|       7604|   Jonathan Hill|      Phoenix|            AZ|           85040|
|       6485|Carolyn Sheppard|Pompano Beach|            FL|           33063|
|       4737|    Mary Mendoza|       Caguas|            PR|             725|

In [14]:
customerDF.select(customerDF['customer_name'],customerDF['customer_city']).show()

+----------------+-------------+
|   customer_name|customer_city|
+----------------+-------------+
|     Mary Torres|       Caguas|
|      Jose Haley|     Columbus|
|      Mary Smith|      Houston|
|  Richard Maddox|       Caguas|
|  Margaret Booth|    Arlington|
|  Mary Henderson|       Caguas|
|     Lisa Walker|       Caguas|
|   Jonathan Hill|      Phoenix|
|Carolyn Sheppard|Pompano Beach|
|    Mary Mendoza|       Caguas|
|   Michael Smith|       Caguas|
|    James Holmes|     Hilliard|
|     Mary Dawson|       Caguas|
|    Adam Marquez|  San Antonio|
|    Gloria Smith|       Caguas|
|       Mary Webb|   San Marcos|
|  Nancy Alvarado|     Flushing|
|  Russell Flores|       Caguas|
|    Denise Smith|    Rego Park|
|  Jose Dickerson|         Mesa|
+----------------+-------------+
only showing top 20 rows



In [17]:
customerDF.select(customerDF['customer_name']).show()

+----------------+
|   customer_name|
+----------------+
|     Mary Torres|
|      Jose Haley|
|      Mary Smith|
|  Richard Maddox|
|  Margaret Booth|
|  Mary Henderson|
|     Lisa Walker|
|   Jonathan Hill|
|Carolyn Sheppard|
|    Mary Mendoza|
|   Michael Smith|
|    James Holmes|
|     Mary Dawson|
|    Adam Marquez|
|    Gloria Smith|
|       Mary Webb|
|  Nancy Alvarado|
|  Russell Flores|
|    Denise Smith|
|  Jose Dickerson|
+----------------+
only showing top 20 rows



In [23]:
customerDF.filter(customerDF['customer_state'] == 'CA').show()

+-----------+----------------+---------------+--------------+----------------+
|customer_id|   customer_name|  customer_city|customer_state|customer_zipcode|
+-----------+----------------+---------------+--------------+----------------+
|       5577|      Mary Smith|        Modesto|            CA|           95350|
|       1745|      Mary Smith|Rowland Heights|            CA|           91748|
|      11444|Kathleen Patrick|      San Diego|            CA|           92109|
|       8846|    Thomas Smith|          Indio|            CA|           92201|
|       6237|  Bobby Anderson|       El Cajon|            CA|           92020|
|       4085|       Mary Carr|  Panorama City|            CA|           91402|
|       8705|  Patricia Smith|       Stockton|            CA|           95207|
|       3669|       Mary Soto| San Bernardino|            CA|           92410|
|       6101|      Mary Smith|    Los Angeles|            CA|           90033|
|      11697|  Jessica Thomas|  Laguna Niguel|      

In [24]:
customerDF.groupBy("customer_state").count().show(10)

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AZ|   19|
|            SC|    2|
|            LA|    7|
|            MN|    1|
|            NJ|   19|
|            DC|    4|
|            OR|    4|
|            VA|   14|
|            RI|    2|
|            KY|    1|
+--------------+-----+
only showing top 10 rows



In [46]:
view = customerDF.groupBy("customer_state").count()

In [47]:
view1 = view.filter(view["count"]>50)

In [33]:
customerDF.createOrReplaceTempView("customerView")

In [35]:
spark.sql("select customer_state , count(*) as cnt  from customerView group by customer_state having cnt > 50 ").show()

+--------------+---+
|customer_state|cnt|
+--------------+---+
|            CA|187|
|            NY| 79|
|            TX| 62|
|            PR|505|
+--------------+---+



In [38]:
view.printSchema()

root
 |-- customer_state: string (nullable = true)
 |-- count: long (nullable = false)



In [49]:
view1.coalesce(1).write.json("/home/hduser/Desktop/view1.json")

In [51]:
productDF = spark.read.load("/home/hduser/Desktop/product.json", format ="json")

In [52]:
productDF.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: long (nullable = true)
 |-- salestxn_id: long (nullable = true)



In [53]:
productDF.show()

+---------------+-----------+-----------+------------------+--------------------+-------------+----------------+-----------+
|_corrupt_record|category_id|customer_id|  product_category|        product_name|product_price|product_quantity|salestxn_id|
+---------------+-----------+-----------+------------------+--------------------+-------------+----------------+-----------+
|           null|         45|      12080|           Fishing|O'Brien Men's Neo...|        49.98|               2|      98660|
|           null|         45|      12279|           Fishing|O'Brien Men's Neo...|        49.98|               2|      97749|
|           null|         12|       3343|      Boxing & MMA|Under Armour Wome...|        31.99|               4|     103889|
|           null|         45|      12382|           Fishing|O'Brien Men's Neo...|        49.98|               5|      22751|
|           null|         47|       5032|           Boating|Pelican Sunstream...|       199.99|               1|     129001|


In [54]:
productDF.select(productDF['product_name'],productDF['product_price']).show()

+--------------------+-------------+
|        product_name|product_price|
+--------------------+-------------+
|O'Brien Men's Neo...|        49.98|
|O'Brien Men's Neo...|        49.98|
|Under Armour Wome...|        31.99|
|O'Brien Men's Neo...|        49.98|
|Pelican Sunstream...|       199.99|
|Nike Men's CJ Eli...|       129.99|
|Diamondback Women...|       299.98|
|Field & Stream Sp...|       399.98|
|Perfect Fitness P...|        59.99|
|Nike Men's CJ Eli...|       129.99|
|Pelican Sunstream...|       199.99|
|Nike Men's CJ Eli...|       129.99|
|Diamondback Women...|       299.98|
|Nike Men's CJ Eli...|       129.99|
|Nike Men's Dri-FI...|         50.0|
|O'Brien Men's Neo...|        49.98|
|O'Brien Men's Neo...|        49.98|
|Nike Men's Dri-FI...|         50.0|
|Diamondback Women...|       299.98|
|Under Armour Girl...|        39.99|
+--------------------+-------------+
only showing top 20 rows



In [57]:
productDF.select(productDF['product_name'],productDF['product_price'],productDF['product_category']).filter(productDF['product_price'] >200 ).show()

+--------------------+-------------+------------------+
|        product_name|product_price|  product_category|
+--------------------+-------------+------------------+
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Field & Stream Sp...|       399.98|Hunting & Shooting|
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Field & Stream Sp...|       399.98|Hunting & Shooting|
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Field & Stream Sp...|       399.98|Hunting & Shooting|
|Field & Stream Sp...|       399.98|Hunting & Shooting|
|Diamondback Women...|       299.98| Bike & Skate Shop|
|Field & Stream Sp...|       399.98|Hunting & Shooting|
|Field & Stream Sp...|       399.98|Hunting & Shooting|
|Field & Stream Sp...|       399.98|Hunting & Sh

In [65]:
view = productDF.groupBy('product_category').count()

In [ ]:
view.